# Introduction to Functional Programming
### Using Python

## Intro

### Lambda
`lambda` is an operator in Python to define anonymous functions inline. The naming is derived from the _lambda calculus_ definition in mathematics, where it stands for a function of computation. As such in python `lambda` is restricted to be without side effects and return primarily return the results of a computation, which it does automatically. Let's take the following example function:

In [3]:
def add_one(x):
    return x + 1

print(add_one(1))
print(add_one(10))

2
11


We could also write this as a `lambda` like this:

In [5]:
add_one = lambda x: x + 1

print(add_one(1))
print(add_one(10))

2
11


`lambda`'s are very handy for functional programming as they allow us to quickly define minor computations inline.

# Examples

## ATM calculator
Let's take the following example of a (pseudo) code to calculate the bills an ATM is supposed to hand out, when a certain amount is requested – which better not be in production anywhere:

In [45]:
def calculate_bills(amount):
    bills = [100, 50, 20, 10, 5, 1]
    results = {}
    for b in bills:
        results[b] = 0
        while amount >= b:
            results[b] += 1
            amount -= b
    return results


In [46]:
calculate_bills(100)

{1: 0, 5: 0, 10: 0, 20: 0, 50: 0, 100: 1}

In [47]:
calculate_bills(80)

{1: 0, 5: 0, 10: 1, 20: 1, 50: 1, 100: 0}

In [48]:
calculate_bills(92)

{1: 2, 5: 0, 10: 0, 20: 2, 50: 1, 100: 0}

### functional Example
We can also write the same function in a more functional style

In [37]:
def calculate_bills(amount):
    def calculate(amount, remaining_bills):
        if not remaining_bills: return []
        bill = remaining_bills[0]
        count = abs(amount / bill)
        return [count] + calculate(amount - (count * bill), remaining_bills[1:])

    bills = [100, 50, 20, 10, 5, 1]
    return dict(zip(bills, calculate(amount, bills)))

In [38]:
calculate_bills(92)

{1: 2, 5: 0, 10: 0, 20: 2, 50: 1, 100: 0}

In [40]:
calculate_bills(109)

{1: 4, 5: 1, 10: 0, 20: 0, 50: 0, 100: 1}

### UFO Data
We have a file `ufo_data.tsv` of about 76Mb here. This is a big chunk of data to go through. Ideally we want to process items one at a time. The following block returns us a `generator` which we can iterate over and does exactly that.

In [91]:
import csv
def read_ufo_data():
    with open("ufo_data.tsv", 'r') as f:
        for row in csv.reader(f, delimiter="\t"):
            yield row
        

In [92]:
r = read_ufo_data()

In [93]:
r

<generator object read_ufo_data at 0x10ae54550>

Until now, we have only opened the file, we haven't even read the first line yet. This will only happen once we access the generator (by calling `next()` on it or iterating over it). These generator or iterator objects allow us to construct half-executed functions. That is called **lazy evulation** as they are only executed when it is needed. 

In [94]:
a = r.next()

In [95]:
a

['19951009',
 '19951009',
 ' Iowa City, IA',
 '',
 '',
 'Man repts. witnessing &quot;flash, followed by a classic UFO, w/ a tailfin at back.&quot; Red color on top half of tailfin. Became triangular.']

Now we have the first record of the file saved in the variable a. We have started **lazy evaluating** the code. To retrieve this record we did not have to load the entire content into memory and as soon as we release `a`, we will free this memory again. This can be very useful in various cases, like here when we are just interested to _find the first sighting in Australia_. **Excercise**: Can you write a short piece of code that finds that?

Remember: the place is listed at index `2` and a `"Australia" in` is totally sufficient here ;) .


In [101]:
filter(lambda x: "Australia" in x[2], read_ufo_data())[0]

['19970206',
 '19970212',
 ' Coober Pedy, S.A. (Approx. 250km south of) (Australia),',
 ' light',
 '',
 'SUMMARY:  A stormy night on the Stuart Highway. Sighting of a UFO which appeared to land,  a UFO chasing the car some 50m behind, and the sighting of a man which was extremely unusual looking.The first event to happen was the sighting of a &apos;roadwork&apos; man sweeping the highway.  Whilst driving along the highway, an extremely bright light suddenly appeared in front of us. At first we thought it was a truck coming towards us - common on the highway-but as we drove towards it we realised it was a stationary truck with a bright light attatched to the front of the grill. Upon getting closer to the light, it was that bright and blinding that we had to slow down to 10kms just to see the road and where we were going. Half the road was closed and suddenly we were able to see a man. There he was sweeping the road at 21:30(approx) 200kms from the nearest roadhouse. We slowly drove up t

## An iterative way
Although our iterator was doing lazy loading, `filter` generate an entire list, even though we only need the first sample. Obviously that is insufficient and we can do better in a more memory efficient manner. Introducing `itertools`. The [itertools](https://docs.python.org/2/library/itertools.html) library is preshipped with all (c)Python versions and contains iterative implementations of `map`, `filter` and such – appropriatly named `imap` and  `ifilter`. Lets use that one:

In [102]:
ifilter(lambda x: "Australia" in x[2], read_ufo_data())

Not only did that evaluate much quicker, as you can see, this creates another `iterator`, which means we are doing a lazy evaluation again. Let's use that to our advantage. To get the next (or in this case _first_) item, we can just call the `next()` function on it. Let's do that:

In [103]:
ifilter(lambda x: "Australia" in x[2], read_ufo_data()).next()

['19970206',
 '19970212',
 ' Coober Pedy, S.A. (Approx. 250km south of) (Australia),',
 ' light',
 '',
 'SUMMARY:  A stormy night on the Stuart Highway. Sighting of a UFO which appeared to land,  a UFO chasing the car some 50m behind, and the sighting of a man which was extremely unusual looking.The first event to happen was the sighting of a &apos;roadwork&apos; man sweeping the highway.  Whilst driving along the highway, an extremely bright light suddenly appeared in front of us. At first we thought it was a truck coming towards us - common on the highway-but as we drove towards it we realised it was a stationary truck with a bright light attatched to the front of the grill. Upon getting closer to the light, it was that bright and blinding that we had to slow down to 10kms just to see the road and where we were going. Half the road was closed and suddenly we were able to see a man. There he was sweeping the road at 21:30(approx) 200kms from the nearest roadhouse. We slowly drove up t

Well, that was much faster!

That is because we have only been reading the file – line by line – to the point that it was necessary. Even better with every line we checked, the reference was discarded and the memory cleared. This kept the footprint of memory very low throughout the entire execution. While also being much faster!

# Conway's Game of Life

A classic computation example for a cool cellular emulation model – which has been studied over and over again – is [conway's game of life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life). In its essence it is based on a matrix of cells, which live through life cycle and which every cycle they life or die depending on a very small set of rules:

 - Any live cell with fewer than two live neighbours dies, as if caused by under-population.
 - Any live cell with two or three live neighbours lives on to the next generation.
 - Any live cell with more than three live neighbours dies, as if by overcrowding.
 - Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

Although these are rather simple, it allows for quite a few awesome thingies which happen:

![](https://upload.wikimedia.org/wikipedia/commons/e/e5/Gospers_glider_gun.gif)

Let's build this out in a purely functional manner:


In [104]:
def is_alive(cell, neighboors):
    alive_neighboors = len(filter(lambda x: x, neighboors))
    if not cell:
        # Any dead cell with exactly three live neighbours becomes a live cell
        return alive_neighboors == 3
    
    if alive_neighboors < 2:
        # Any live cell with fewer than two live neighbours dies,
        return False
    
    if alive_neighboors > 3:
        # Any live cell with more than three live neighbours dies,
        return False
    
    # Any live cell with two or three live neighbours lives on to the next generation
    return True

So the core of our algorithm is this decision maker. It takes a cell, represented with a truthy or falsy value representing its current state (`cell`) and a list of its neighboors, also represented through their current state as a truthy (_alive_) or falsy (_dead_) value.

# ---- STUFF BROKEN

Given a grid as a list of values, and the number of cols and rows this list represents, we need to map the alive function over each and every cell. In order to be able to do that, we need to map over all cells and find its neighboors and then call is_alive on the cell. Alright, let's go:

In [244]:
def next_cycle(matrix):
    def safe_get_at_index(row, col):
        # a helper because for borders of the matrix, python throws exceptions
        try:
            return matrix[row][col]
        except IndexError:
            return False

    # a helper which matches safe_get_at_index over all surrounding elements for a given col, row
    get_neighboors = lambda row, col: map(lambda x: safe_get_at_index(row + x[0], col + x[1]), [
                    (-1, -1),
                    (-1, 0),
                    (-1, 1),
                    (0, -1),
                    (0, 1),
                    (1, -1),
                    (1, 0),
                    (1, 1)
                   ])
    
    print get_neighboors(1,1)
    print get_neighboors(4,4)
    get_next_state = lambda row, col: is_alive(matrix[row][col], get_neighboors(row, col))
    return map(lambda idx: get_next_state(idx[0], idx[1]),
                   [ [(row_idx, x) for x in xrange( len( matrix[row_idx] ) ) ]
                        for row_idx in xrange( len( matrix ) ) ]
               )


In [237]:
def print_matrix(matrix):
    for row in matrix:
        print("".join([cell and '#' or '·' for cell in row]))
            

In [238]:
import random
def random_matrix(cols=10, rows=10, prob=0.3):
    for c in xrange(rows):
        yield [random.random() <= prob for r in xrange(cols)]
            

In [239]:
matrix = [x for x in random_matrix(10, 10)]

In [240]:
print_matrix(matrix)

···##·····
··##····#·
·#···###··
#····#··#·
·#······#·
#··#······
··#·······
#······#··
#······#··
#··##···#·


In [245]:
for x in xrange(10):
    matrix = next_cycle(matrix)
    print(" ------------------- {}".format(x))
    print_matrix(matrix)

[False, False, False, False, True, False, True, False]
[False, False, True, False, False, True, False, False]


TypeError: list indices must be integers, not tuple